In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
from matplotlib import pyplot as plt

## Statistical Power

Consider a previous example 

* $H_0$ : Karen can't correctly guess the weather more than 30% of the time.
* $H_a$ : Karen can correctly guess the weather more than 30% of the time.

What if $H_0$ is true, but we incorrectly reject it?
What if $H_a$ is true, but we incorrectly fail to reject $H_0$?

x| H0 is True            | H0 is False
-------------|----------------------|------------------
**Accept H0**| Correct ($1 - \alpha$)   | Type II error ($\beta$)
**Reject H0** | Type I error ($\alpha$) | Correct ($1 - \beta$)


* alpha is the type I error, also known as false positive rate. It's the probability that we incorrectly reject the null hypothesis.

* beta is the Type II error, also known as false negative rate. It's the probability that we incorrectly fail to reject the null hypothesis.

We *choose* $\alpha$ - this corresponds to our confidence level.

What might affect $\beta$?

<!-- N is the main piece -->

In [ ]:
# set up my subplots
fig,axes = plt.subplots(3, figsize = (8, 12))

# for each subplot we're going to iterate through and capture the index and the plot object
for ax_idx, ax in enumerate(axes):

    # Setting up the n and ps for the distributions. 
    # We want to vary the actual value for Karen's correct guessing rate. (not the sampled value)
    n=100
    null_proportion = 0.3
    karen_proportion = 0.29 + 0.05 * ax_idx

    # sampling distribution
    # Stating that karen's true guessing rate should be the sample mean
    karen_sample_mean = karen_proportion
    
    # Establish the standard deviation for Karen's guessing rate
    karen_sd = np.sqrt(karen_proportion * (1-karen_proportion))
    
    # This is the standard error for Karen's sample mean.
    karen_sample_mean_sd = karen_sd / np.sqrt(n)

    # Create a normal distribution for Karen's sample mean. 
    karen = stats.norm(karen_sample_mean, karen_sample_mean_sd)

    # Create sample space of a 100 points between 0.1 and 0.6
    support = np.linspace(0, 1, 100)
    # Finding the pdf at each of these points.
    karen_pdf = karen.pdf(support)

    # Finding the number where we reject the null hypothesis.
    null_mean = 0.3
    null_sd = np.sqrt((null_proportion * (1-null_proportion))/n)
    critical_value_dist = stats.norm(0.3, null_sd)
    critical_value = critical_value_dist.ppf(0.95)
    # find the inddex of the support where it becomes greater than the critical value (threshold)
    critical_value_idx = np.argmax(support>critical_value)
    
    # Plots
    ax.plot(support, karen_pdf, color='blue', label='Karen')
    # axvline is for a vertical line
    ax.axvline(critical_value, label = f'{round(critical_value,2)}')

    ax.fill_between(support, karen_pdf, \
                    where = (support >= critical_value), label = 'Power', \
                   color = 'grey')

    # legend and title
    ax.legend()
    ax.set_title("Karen's Actual %Correctly Guessing: {:.2f}".format(karen_proportion))

# Effect Size

The difference between the mean assumed by the null hypothesis and the **true** mean is the effect size. The larger this is, the more likely we are to draw a sample which meets the threshold required by our $\alpha$

#### With a larger effect size, we can get more power with the same alpha.

In [ ]:

fig,axes = plt.subplots(3, figsize = (8, 12))

for ax_idx, ax in enumerate(axes):
    n=100
    karen_proportion = 0.29 + 0.05 * ax_idx

    # sampling distribution
    karen_sample_mean = karen_proportion
    karen_sd = np.sqrt(karen_proportion * (1-karen_proportion))
    karen_sample_mean_sd = karen_sd / np.sqrt(n)


    karen = stats.norm(karen_sample_mean, karen_sample_mean_sd)
    null = stats.norm(0.3, np.sqrt(0.3 * 0.7 / n))
    
    support = np.linspace(0.1, 0.6, 100)
    karen_pdf = karen.pdf(support)
    null_pdf = null.pdf(support)
    
    
    critical_value = stats.norm(0.3, np.sqrt((null_proportion * (1-null_proportion))/n)).ppf(0.95)
    critical_value_idx = np.argmax(support>critical_value)
    
    ax.plot(support, karen_pdf, color='blue')
    ax.plot(support, null_pdf, color='red')

    ax.axvline(critical_value)
    ax.fill_between(support, karen_pdf, where = (support >= critical_value), label = 'Power', color = 'grey', alpha = 0.5)
    ax.fill_between(support, karen_pdf, where = (support <= critical_value), alpha = 0.5, label = 'beta', color = 'blue')
    ax.fill_between(support, null_pdf, where = (support >= critical_value), label = 'alpha', color = 'red', alpha = 0.5)
    
    point2 = [karen_sample_mean, 8.7]
    point1 = [0.3, 8.7]
    x_vals = [point1[0], point2[0]]
    y_vals = [point1[1], point2[1]]
    
    ax.plot(x_vals, y_vals, c = 'k', label = 'Effect Size', ls = '--')
    ax.legend()
    ax.set_title("Effect Size: {:.2f}".format(karen_proportion - 0.3))


#### With a higher sample size, we get more power.

In [ ]:

fig,axes = plt.subplots(3, figsize = (8, 12))

for ax_idx, ax in enumerate(axes):
    n= 100 * 2 ** ax_idx
    karen_proportion = 0.33

    # sampling distribution
    karen_sample_mean = karen_proportion
    karen_sd = np.sqrt(karen_proportion * (1-karen_proportion))
    karen_sample_mean_sd = karen_sd / np.sqrt(n)


    karen = stats.norm(karen_sample_mean, karen_sample_mean_sd)
    null = stats.norm(0.3, np.sqrt(0.3 * 0.7 / n))
    
    support = np.linspace(0.1, 0.6, 100)
    karen_pdf = karen.pdf(support)
    null_pdf = null.pdf(support)
    
    critical_value = stats.norm(0.3, np.sqrt((null_proportion * (1-null_proportion))/n)).ppf(0.95)
    critical_value_idx = np.argmax(support>critical_value)
    
    ax.plot(support, karen_pdf, color='blue')
    ax.plot(support, null_pdf, color='red')

    ax.axvline(critical_value)
    
    ax.fill_between(support, karen_pdf, where = (support >= critical_value), label = 'Power', color = 'grey', alpha = 0.5)
    ax.fill_between(support, karen_pdf, where = (support <= critical_value), alpha = 0.5, label = 'beta', color = 'blue')
    ax.fill_between(support, null_pdf, where = (support >= critical_value), label = 'alpha', color = 'red', alpha = 0.5)
    
    ax.legend()
    ax.set_title("Sample Size: {:.2f}".format(n))


## Power is a function of sample size, effect size, alpha, and standard deviation

$$ n = \left( \frac{ \sigma ( F_{0, 1}^{-1} (1 - \alpha) - F_{0, 1}^{-1} (\beta) ) } { \mu_a - \mu_0 } \right)^2 $$




$F_{0, 1}^{-1}$ is equivalent to `stats.norm(0,1).ppf()`

This is the formula for the sample size needed to achieve a given power $(1 - \beta)$.

$\sigma$ is the standard deviation

$\mu_a - \mu_0$ is the effect size. (mean under alternative vs mean under null).

$\alpha$ is maximum acceptable false positive rate aka incorrectly rejecting null hypothesis. $\alpha$

$\beta$ is maximum acceptable fals negative rate, aka incorrectly failing to reject null hypothesis

$n$ is the sample size

In [ ]:
# Spend some time coding this up

def sample_size_needed_for_power(sigma, alpha, power, effect_size):
    standard_normal = stats.norm(0, 1)
    beta = 1 - power
    numerator = sigma * (standard_normal.ppf(1 - alpha) - standard_normal.ppf(beta))
    return int((numerator / effect_size) ** 2)

# Power calculation examples

### CTR
Our current click through rate for our ads is 2%. We'd like to try a new ad and if it's better, we'll put it into production. We want to be able to detect a bump of at least a tenth of a point. We'd like to be 95% confident that the new ad is better than 2%. If the new ad actually is better, we want at least an 80% chance of knowing that. How many impressions do we need?

$H_0$:  <br>
$H_a$: 

effect size:

alpha: 

power:  

sigma: 

<!-- sigma = np.sqrt(p*1-p) for proportions since we dont need count -->

In [ ]:
sample_size_needed_for_power(sigma = , alpha = , power, effect_size = ) 

What can you tell your product manager?

### Student Assessment scores
At Galvanize, students get on average, 65% on assessments (*this is made up, I have no idea*). A new instructor has been hired, and we want to know if they are adequately teaching their students. The standard deviation for assessment scores is 3 points. We'd like to be able to detect if the average score is 60% or less, with a significance level of 0.01, and a power of 0.9. How many student assessment scores do we need to see to make this determination?


$H_0$:  <br>
$H_a$: 

effect size: 

alpha: 

power: 

sigma: 

In [ ]:
sample_size_needed_for_power(sigma = , alpha = , power= , effect_size = ) 

# MURDER MOST FOUL

We have a bunch of horror movie characters who have all been convicted of murders.

Investigators on average solve 80% of murders.
When they use DNA tests, are they better at solving murders?

$H_0$: DNA testing does not give us better than 80% solve rate. <br>
$H_a$: DNA testing gives us better than 80% solve rate

Type I Error:  
Type II Error: 

alpha: 

beta:  


If we assume Type I Error is worse than Type II, what should we set for alpha and beta. 

If the actual solve rate using DNA testing is at least 85% we want to be able to detect that. 

Sample size is how many cases we need to look at to determine whether or not DNA evidence is useful. 

* alpha = 
* beta = 
* effect_size = 
* sigma = 



In [ ]:
sample_size_needed_for_power(sigma = , alpha = , power= , effect_size = ) 

### Why is the proportion standard deviation sqrt(pq)?

The population is distributed as a Bernoulli distribution. Positive case is '1' and negative case is '0'. Every individual sampled from the population is a sample from a Bernoulli distribution. The standard deviation for the Bernoulli distribution is sqrt(pq). 